In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd '/content/drive/MyDrive/Waterloo/_MSCI_641/project'

/content/drive/MyDrive/Waterloo/_MSCI_641/project


In [4]:
!pip install virtualenv
!pip install rouge_score
!pip install transformers
!pip install wandb
!pip install datasets
!pip install evaluate
!pip install accelerate
# !pip install gradio --upgrade
# !pip install --upgrade torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 46.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=70f3bf86ee5af541b5cf473c60fd3b9fb44204497738a45022ee3f577482b37d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [5]:
import numpy as np
import pandas as pd
import json, os, gc

import evaluate
import torch, wandb
from tqdm.auto import tqdm
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import default_data_collator
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModelForSequenceClassification, AdamW, RobertaTokenizer
from collections import defaultdict
from transformers import Trainer, TrainingArguments
from accelerate import Accelerator
from transformers import get_scheduler
# import gradio as gr
from utils import *
from sklearn.metrics import f1_score, accuracy_score

os.environ["TOKENIZERS_PARALLELISM"] = "false"

tqdm.pandas()

In [6]:
datasets = dict()
dataset_names = ['train','val','test']

for dataset_name in dataset_names:
    datasets[dataset_name] = pd.read_json(path_or_buf=dataset_name + '.jsonl', lines=True)
    if 'id' not in datasets[dataset_name].columns:
        datasets[dataset_name]['id'] = datasets[dataset_name].index

features = ['postText', 'postPlatform', 'targetParagraphs', 'targetTitle',
       'targetDescription', 'targetKeywords', 'targetMedia', 'targetUrl']
features = ['postText', 'postPlatform', 'targetParagraphs', 'targetTitle',
       'targetDescription', 'targetKeywords', 'targetMedia', 'targetUrl']
features_text = ['postText','targetParagraphs', 'targetTitle', 'targetDescription',
                'targetKeywords']
features_url = ['targetUrl', 'targetMedia' ]

target1 = 'tags'
target2 = 'spoiler'

In [7]:
label_encoder = {"phrase": 0,"passage": 1,"multi":2}
label_decoder = {0:"phrase",1:"passage", 2:"multi"}

for dataset_name in dataset_names:

    print(dataset_name)

    temp_data = datasets[dataset_name]

    if dataset_name != 'test':
        temp_data = temp_data[temp_data['spoilerPositions'].apply(lambda x: len(x[0])) > 1]
        temp_data['tokPos'] = temp_data.apply(findPosTags,1)

        temp_data['tags'] = temp_data['tags'].apply(lambda x: x[0])
        temp_data['label'] = temp_data['tags'].map(label_encoder)
        temp_data['tokenPosition'] = temp_data.apply(findPosTags,1)
        temp_data['longest_spoiler_index'] = temp_data['spoiler'].apply(return_longest_spoiler_index)
        temp_data['longest_spoiler'] = temp_data.apply(lambda x: x['spoiler'][x['longest_spoiler_index']],1)
        temp_data['longest_tokPos'] = temp_data.apply(lambda x: x['tokPos'][x['longest_spoiler_index']],1)

    temp_data['postText'] = temp_data['postText'].apply(lambda x: x[0])

    for col in ['targetParagraphs', 'targetMedia']:

        print(col)

        temp_data[col + '_listlen'] = temp_data[col].apply(lambda x: len(x) if x is not None else 0)
        temp_data[col + '_concat'] = temp_data[col].apply(lambda x: ' '.join(x) if x is not None else '')

    for col in ['postText', 'targetTitle', 'targetDescription', 'targetKeywords', 'targetParagraphs_concat']:

        print(col)
        temp_data[col + '_len'] = temp_data[col].apply(lambda x: len(x.split()) if x is not None else 0)

    temp_data['title_para'] = temp_data['targetTitle'] + " - " + temp_data['targetParagraphs_concat']

    datasets[dataset_name] = temp_data

train
targetParagraphs


<ipython-input-7-0d1bb7c1b72f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['tokPos'] = temp_data.apply(findPosTags,1)
<ipython-input-7-0d1bb7c1b72f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['tags'] = temp_data['tags'].apply(lambda x: x[0])
<ipython-input-7-0d1bb7c1b72f>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

targetMedia
postText
targetTitle
targetDescription
targetKeywords
targetParagraphs_concat
val
targetParagraphs
targetMedia
postText
targetTitle
targetDescription
targetKeywords
targetParagraphs_concat
test
targetParagraphs
targetMedia
postText
targetTitle
targetDescription
targetKeywords
targetParagraphs_concat


<ipython-input-7-0d1bb7c1b72f>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data[col + '_len'] = temp_data[col].apply(lambda x: len(x.split()) if x is not None else 0)
<ipython-input-7-0d1bb7c1b72f>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['title_para'] = temp_data['targetTitle'] + " - " + temp_data['targetParagraphs_concat']


In [8]:
datasets['test'].iloc[8]

postId                                                                    53e76y
postText                       The First Person In The Royal Family Has Just ...
postPlatform                                                              reddit
targetParagraphs               [The Queen’s cousin has become the first membe...
targetTitle                    The first person in the royal family has just ...
targetDescription              The Queen’s cousin has become the first member...
targetKeywords                                                              None
targetMedia                    [http://cdn.pinknews.co.uk/images/2016/08/truv...
targetUrl                      http://www.pinknews.co.uk/2016/09/18/the-first...
id                                                                             8
targetParagraphs_listlen                                                      18
targetParagraphs_concat        The Queen’s cousin has become the first member...
targetMedia_listlen         

In [9]:
datasets['train']['tags']

0       passage
1        phrase
2        phrase
3         multi
4        phrase
         ...   
3195    passage
3196    passage
3197      multi
3198     phrase
3199     phrase
Name: tags, Length: 3199, dtype: object

In [10]:
datasets['train'][['spoiler','tokPos','longest_spoiler', 'spoilerPositions','longest_tokPos']]

,spoiler,tokPos,longest_spoiler,spoilerPositions,longest_tokPos
0,[how about that morning we go throw?],"[[772, 807]]",how about that morning we go throw?,"[[[3, 151], [3, 186]]]","[772, 807]"
1,[2070],"[[67, 71]]",2070,"[[[0, 0], [0, 4]]]","[67, 71]"
2,[intellectual stimulation],"[[349, 373]]",intellectual stimulation,"[[[1, 186], [1, 210]]]","[349, 373]"
3,[Purpose connects us to something bigger and i...,"[[2360, 2436], [3808, 3837], [5148, 5214], [63...",Purpose connects us to something bigger and in...,"[[[11, 25], [11, 101]], [[17, 56], [17, 85]], ...","[2360, 2436]"
4,[in a rice cooker],"[[769, 785]]",in a rice cooker,"[[[5, 60], [5, 76]]]","[769, 785]"
...,...,...,...,...,...
3195,[it hasn’t necessarily taken the wind out of Y...,"[[1834, 1902]]",it hasn’t necessarily taken the wind out of Yo...,"[[[7, 50], [7, 118]]]","[1834, 1902]"
3196,"[It read, ""Thanks for keeping us safe.""]","[[416, 454]]","It read, ""Thanks for keeping us safe.""","[[[0, 317], [0, 355]]]","[416, 454]"
3197,[1. Anti-wrinkle creams will erase the fine li...,"[[873, 946], [1441, 1550], [2266, 2329], [3039...",2. Dark circles will appear beneath your eyes ...,"[[[6, 0], [6, 73]], [[10, 0], [10, 109]], [[14...","[1441, 1550]"
3198,[@beyoncefan666],"[[495, 509]]",@beyoncefan666,"[[[3, 55], [3, 69]]]","[495, 509]"


In [11]:
datasets['train'].columns

Index(['uuid', 'postId', 'postText', 'postPlatform', 'targetParagraphs',
       'targetTitle', 'targetDescription', 'targetKeywords', 'targetMedia',
       'targetUrl', 'provenance', 'spoiler', 'spoilerPositions', 'tags', 'id',
       'tokPos', 'label', 'tokenPosition', 'longest_spoiler_index',
       'longest_spoiler', 'longest_tokPos', 'targetParagraphs_listlen',
       'targetParagraphs_concat', 'targetMedia_listlen', 'targetMedia_concat',
       'postText_len', 'targetTitle_len', 'targetDescription_len',
       'targetKeywords_len', 'targetParagraphs_concat_len', 'title_para'],
      dtype='object')

In [12]:
bleu = evaluate.load("bleu")
squad_metric = evaluate.load("squad")

In [13]:
meteor = evaluate.load("meteor")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [14]:
rouge = evaluate.load("rouge")

In [15]:
train_df = convert2squadFormat(datasets['train'], 'train')
val_df = convert2squadFormat(datasets['val'], 'val')
test_df = convert2squadFormat(datasets['test'], 'test')

train_data = Dataset.from_pandas(train_df.reset_index(drop=True), split="train")
val_data = Dataset.from_pandas(val_df.reset_index(drop=True), split="val")
test_data = Dataset.from_pandas(test_df.reset_index(drop=True), split="test")

/content/drive/MyDrive/Waterloo/_MSCI_641/project/utils.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fin["asnwers"] = df_fin.apply(lambda x: {'text':[x['longest_spoiler']], "answer_start":[x['longest_tokPos'][0]]},1)
/content/drive/MyDrive/Waterloo/_MSCI_641/project/utils.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fin["asnwers"] = df_fin.apply(lambda x: {'text':[x['longest_spoiler']], "answer_start":[x['longest_tokPos'][0]]},1)
/content/drive/MyDrive/Waterloo/_MSCI_641/project/utils.

In [51]:
val_df

,id,labels,title,question,context,answers
0,0,1,Five Nights at Freddy’s Sequel Delayed for Wei...,Five Nights at Freddy’s Sequel Delayed for Wei...,Five Nights at Freddy’s Sequel Delayed for Wei...,{'text': ['some of the plot elements are so di...
1,1,2,Why Arizona Sheriff Joe Arpaio’s fate could ha...,Why Arizona Sheriff Joe Arpaio’s fate could ha...,Why Arizona Sheriff Joe Arpaio’s fate could ha...,{'text': ['could transform a court case agains...
2,2,0,Here’s how much you should be tipping your hai...,Here’s how much you should be tipping your hai...,Here’s how much you should be tipping your hai...,"{'text': ['20%'], 'answer_start': [729]}"
3,3,2,"Alan Rickman & Rupert Grint On 'CBGB,' Reuniti...","""Harry Potter"" alums reunite for new movie","Alan Rickman & Rupert Grint On 'CBGB,' Reuniti...","{'text': ['Alan Rickman & Rupert Grint'], 'ans..."
4,4,1,Man swallowed a microSD card and you won't bel...,A man swallowed a microSD card and you won't b...,Man swallowed a microSD card and you won't bel...,{'text': ['a man who swallowed a 64GB microSD ...
...,...,...,...,...,...,...
395,395,1,Tragic Details Of Kimbo Slice's Death Have Bee...,Tragic Details Of Kimbo Slice's Death Have Bee...,Tragic Details Of Kimbo Slice's Death Have Bee...,"{'text': ['congestive heart failure, along wit..."
396,396,2,Boeing is making a major change to its planes ...,Boeing is making a major change to its planes ...,Boeing is making a major change to its planes ...,{'text': ['increased the air pressure inside t...
397,397,0,Here’s Why Blizzard Is Already Banning Thousan...,Here’s Why Blizzard Is Already Banning Thousan...,Here’s Why Blizzard Is Already Banning Thousan...,"{'text': ['cheat programs'], 'answer_start': [..."
398,398,2,Nine of the weirdest Easter traditions around ...,Nine of the weirdest Easter traditions happeni...,Nine of the weirdest Easter traditions around ...,"{'text': ['Inter-church rocket war: Chios'], '..."


In [16]:
train_df

,id,labels,title,question,context,answers
0,0,1,"Wes Welker Wanted Dinner With Tom Brady, But P...","Wes Welker Wanted Dinner With Tom Brady, But P...","Wes Welker Wanted Dinner With Tom Brady, But P...",{'text': ['how about that morning we go throw?...
1,1,0,Hole In Ozone Layer Expected To Make Full Reco...,NASA sets date for full recovery of ozone hole,Hole In Ozone Layer Expected To Make Full Reco...,"{'text': ['2070'], 'answer_start': [67]}"
2,2,0,Intellectual Stimulation Trumps Money For Empl...,This is what makes employees happy -- and it's...,Intellectual Stimulation Trumps Money For Empl...,"{'text': ['intellectual stimulation'], 'answer..."
3,3,2,"‘Follow your passion’ is wrong, here are 7 hab...",Passion is overrated — 7 work habits you need ...,"‘Follow your passion’ is wrong, here are 7 hab...",{'text': ['Purpose connects us to something bi...
4,4,0,Revealed: The perfect way to cook rice so that...,The perfect way to cook rice so that it's perf...,Revealed: The perfect way to cook rice so that...,"{'text': ['in a rice cooker'], 'answer_start':..."
...,...,...,...,...,...,...
3195,3195,1,"Facebook Video Surging, But YouTube Still Offe...",Has Facebook's video explosion completely shak...,"Facebook Video Surging, But YouTube Still Offe...",{'text': ['it hasn’t necessarily taken the win...
3196,3196,1,Cop is eating at Chili's when teen hands him f...,Cop Is Eating At A Chili's When Teen Hands Him...,Cop is eating at Chili's when teen hands him f...,"{'text': ['It read, ""Thanks for keeping us saf..."
3197,3197,2,5 popular myths about visible signs of aging t...,5 popular myths about visible signs of aging t...,5 popular myths about visible signs of aging t...,{'text': ['2. Dark circles will appear beneath...
3198,3198,0,"WTF, It Looks Like This Twitter Account ""Predi...",You need to see this Twitter account that pred...,"WTF, It Looks Like This Twitter Account ""Predi...","{'text': ['@beyoncefan666'], 'answer_start': [..."


In [32]:
model_name = "csarron/bert-base-uncased-squad-v1"

model_name = "deepset/minilm-uncased-squad2"

model_name = "deepset/roberta-base-squad2"

model_name = "FacebookAI/roberta-base"

model_name = "deepset/roberta-base-squad2"


# Phrase Generation
config = dict(
max_length = 512,
stride = 128,
n_best = 10,
max_answer_length = 30,
batch_size = 16,
epochs = 10,
learning_rate = 1e-6,
model_name = model_name,
)


max_length = config["max_length"]
stride = config["stride"]
n_best = config["n_best"]
max_answer_length = config["max_answer_length"]


tokenizer = AutoTokenizer.from_pretrained(config["model_name"])
#model = AutoModelForQuestionAnswering.from_pretrained(config["model_name"])

In [33]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, RobertaPreTrainedModel, RobertaModel, AdamW, get_linear_schedule_with_warmup
from transformers.modeling_outputs import QuestionAnsweringModelOutput
from datasets import load_dataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


class RobertaForQuestionAnsweringWithClassification(RobertaPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.roberta = RobertaModel(config)
        self.qa_outputs = nn.Linear(config.hidden_size, 2)
        self.classifier = nn.Linear(config.hidden_size, 3)  # Binary classification head

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        start_positions=None,
        end_positions=None,
        labels=None
    ):
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        sequence_output = outputs[0]

        logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        pooled_output = outputs[1]
        classification_logits = self.classifier(pooled_output)

        total_loss = None
        cls_loss = None
        qa_loss = None
        if start_positions is not None and end_positions is not None:
            if len(start_positions.size()) > 1:
                start_positions = start_positions.squeeze(-1)
            if len(end_positions.size()) > 1:
                end_positions = end_positions.squeeze(-1)
            ignored_index = start_logits.size(1)
            start_positions = start_positions.clamp(0, ignored_index)
            end_positions = end_positions.clamp(0, ignored_index)

            loss_fct = nn.CrossEntropyLoss(ignore_index=ignored_index)
            start_loss = loss_fct(start_logits, start_positions)
            end_loss = loss_fct(end_logits, end_positions)
            qa_loss = (start_loss + end_loss) / 2

            if labels is not None:
                cls_loss_fct = nn.CrossEntropyLoss()
                cls_loss = cls_loss_fct(classification_logits, labels)
                total_loss = qa_loss + cls_loss
            else:
                total_loss = qa_loss

        return {'loss':total_loss, 'start_logits':start_logits,
                'end_logits':end_logits, 'hidden_states':outputs.hidden_states,
                'qa_loss':qa_loss,
                'cls_loss':cls_loss,
                'attentions':outputs.attentions,
                'classification_logits':classification_logits}

        return QuestionAnsweringModelOutput(
            loss=total_loss,
            start_logits=start_logits,
            end_logits=end_logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions
        )

In [34]:
model = RobertaForQuestionAnsweringWithClassification.from_pretrained(config["model_name"])
tokenizer = AutoTokenizer.from_pretrained(config["model_name"])


Some weights of RobertaForQuestionAnsweringWithClassification were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['classifier.bias', 'classifier.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:

def preprocess_training_examples(examples):
    questions = examples['question']
    tags = examples['labels']
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    final_tags = []

    for i, offset in enumerate(inputs["offset_mapping"]):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        tag = tags[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        final_tags.append(tag)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    inputs['labels'] = final_tags
    return inputs

In [36]:

def preprocess_test_examples(examples):
    questions = examples["question"]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []
    print('len(inputs["input_ids"])',len(inputs["input_ids"]))

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids

    return inputs


In [37]:
train_dataset = train_data.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=train_data.column_names)

Map:   0%|          | 0/3199 [00:00<?, ? examples/s]

In [38]:
validation_dataset = val_data.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=val_data.column_names,
)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [39]:
test_dataset = test_data.map(
    preprocess_test_examples,
    batched=True,
    remove_columns=test_data.column_names,
)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

len(inputs["input_ids"]) 849


In [40]:
train_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask', 'offset_mapping', 'example_id', 'start_positions', 'end_positions'],
    num_rows: 6578
})

In [41]:
test_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 849
})

In [42]:
validation_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask', 'offset_mapping', 'example_id', 'start_positions', 'end_positions'],
    num_rows: 863
})

In [43]:
len(train_dataset), len(validation_dataset), len(test_dataset)

(6578, 863, 849)

In [44]:
train_set = train_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
test_set = test_dataset.remove_columns(["example_id", "offset_mapping"])
train_set.set_format("torch")
validation_set.set_format("torch")
test_set.set_format("torch")

train_dataloader = DataLoader(
    train_set,
    collate_fn=default_data_collator,
    batch_size=config["batch_size"],
)

eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=32
)
test_dataloader = DataLoader(
    test_set, collate_fn=default_data_collator, batch_size=32
)

optimizer = AdamW(model.parameters(), lr=config["learning_rate"])
accelerator = Accelerator(mixed_precision ='fp16')
model, optimizer, train_dataloader, eval_dataloader, test_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader, test_dataloader
)

num_train_epochs = config["epochs"]
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [45]:


def compute_metrics(start_logits, end_logits,class_logits, features, examples, predictOnly=False):
    example_to_features = defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_spoilers = []
    predicted_classes = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        spoilers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                        'class': class_logits[feature_index].argmax()
                    }
                    spoilers.append(answer)

        # Select the answer with the best score
        if len(spoilers) > 0:
            best_answer = max(spoilers, key=lambda x: x["logit_score"])
            predicted_spoilers.append(
                {"id": example_id, "prediction_text": best_answer["text"]})
            predicted_classes.append(best_answer['class'])
        else:
            predicted_spoilers.append({"id": example_id, "prediction_text": ""})
            predicted_classes.append(0)

    predicted_texts = [i['prediction_text'] for i in predicted_spoilers]

    if predictOnly:
        return predicted_texts, predicted_classes

    actual_spoilers_squad = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    actual_spoilers = [i['answers']['text'][0] for i in actual_spoilers_squad]
    actual_class = [ex['labels'] for ex in examples]

    squad_metrics_eval = squad_metric.compute(predictions=predicted_spoilers, references=actual_spoilers_squad)
    bleu_eval = bleu.compute(predictions=predicted_texts, references=actual_spoilers)
    meteor_eval = meteor.compute(predictions=predicted_texts, references=actual_spoilers)
    rouge_eval = rouge.compute(predictions=predicted_texts, references=actual_spoilers)
    class_f1 = f1_score(actual_class, predicted_classes, average='weighted')
    class_acc = accuracy_score(actual_class, predicted_classes)
    other_metrics = {'f1':class_f1, 'accuracy':class_acc}

    metrics = {
        'squad_exact_match':squad_metrics_eval['exact_match'],
        'squad_f1':squad_metrics_eval['f1'],
        'bleu':bleu_eval['bleu'],
        'meteor':meteor_eval['meteor'],
        'rouge1':rouge_eval['rouge1'],
        'rouge2':rouge_eval['rouge2'],
        'rougeL':rouge_eval['rougeL'],
        'rougeLsum':rouge_eval['rougeLsum'],
        'class_accuracy':class_acc,
        'class_f1':class_f1
        }

    for k,v in metrics.items():
        metrics[k] = np.round(v, 4)

    return metrics, actual_spoilers,predicted_texts, predicted_classes


In [46]:
stats = pd.DataFrame()

for epoch in range(num_train_epochs):
    print("Starting epoch: ", epoch)
    # Training
    model.train()
    train_loss = 0
    train_cls_loss = 0
    train_qa_loss = 0
    start_logits_t = []
    end_logits_t = []
    class_logits_t = []
    for batch in tqdm(train_dataloader):
        outputs = model(**batch)
        loss = outputs['loss']
        qa_loss = outputs['qa_loss']
        cls_loss = outputs['cls_loss']
        accelerator.backward(loss)
        train_loss+=loss.item()
        train_qa_loss+=qa_loss.item()
        train_cls_loss+=cls_loss.item()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        start_logits_t.append(accelerator.gather(outputs['start_logits']).detach().cpu().numpy())
        end_logits_t.append(accelerator.gather(outputs['end_logits']).detach().cpu().numpy())
        class_logits_t.append(accelerator.gather(outputs['classification_logits']).detach().cpu().numpy())

    start_logits = np.concatenate(start_logits_t)
    end_logits = np.concatenate(end_logits_t)
    class_logits = np.concatenate(class_logits_t)
    start_logits = start_logits[: len(train_dataset)]
    end_logits = end_logits[: len(train_dataset)]
    class_logits = class_logits[: len(train_dataset)]

    metrics,theoretical_texts,predicted_texts, predicted_classes = compute_metrics(
        start_logits, end_logits,class_logits, train_dataset, train_data
    )
    del start_logits, end_logits, class_logits,start_logits_t, end_logits_t,class_logits_t, outputs, batch
    torch.cuda.empty_cache()

    train_metrics = {'train_' + k:v for k,v in metrics.items()}

    # Evaluation
    model.eval()
    start_logits_t = []
    end_logits_t = []
    class_logits_t = []
    accelerator.print("Evaluation!")

    val_loss = 0
    val_qa_loss = 0
    val_cls_loss = 0
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)
            val_loss += outputs['loss'].item()
            val_qa_loss += outputs['qa_loss'].item()
            val_cls_loss += outputs['cls_loss'].item()

        start_logits_t.append(accelerator.gather(outputs['start_logits']).cpu().numpy())
        end_logits_t.append(accelerator.gather(outputs['end_logits']).cpu().numpy())
        class_logits_t.append(accelerator.gather(outputs['classification_logits']).cpu().numpy())

    start_logits = np.concatenate(start_logits_t)
    end_logits = np.concatenate(end_logits_t)
    class_logits = np.concatenate(class_logits_t)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]
    class_logits = class_logits[: len(validation_dataset)]

    metrics,theoretical_texts,predicted_texts, predicted_classes = compute_metrics(
        start_logits, end_logits,class_logits, validation_dataset, val_data
    )

    val_metrics = {'val_' + k:v for k,v in metrics.items()}

    metrics = train_metrics
    for k,v in val_metrics.items():
      metrics[k] = v

    metrics['epoch'] = epoch

    train_loss = train_loss/len(train_dataloader)
    val_loss = val_loss/len(eval_dataloader)
    metrics['train_loss'] = train_loss
    metrics['val_loss'] = val_loss

    train_qa_loss = train_qa_loss/len(train_dataloader)
    val_qa_loss = val_qa_loss/len(eval_dataloader)
    metrics['train_qa_loss'] = train_qa_loss
    metrics['val_qa_loss'] = val_qa_loss

    train_cls_loss = train_cls_loss/len(train_dataloader)
    val_cls_loss = val_cls_loss/len(eval_dataloader)
    metrics['train_cls_loss'] = train_cls_loss
    metrics['val_cls_loss'] = val_cls_loss

    print("Epoch stats: ")
    print(metrics)

    df_metrics = pd.DataFrame(metrics.values()).T
    df_metrics.columns=metrics.keys()
    stats = pd.concat([stats,df_metrics])

    stats.to_csv("results_both_robertasquad_new/metrics2.csv")

#    model.save_pretrained(f"./models_task2_/")

Starting epoch:  0


  0%|          | 0/412 [00:00<?, ?it/s]

  0%|          | 0/3199 [00:00<?, ?it/s]

Evaluation!


  0%|          | 0/27 [00:00<?, ?it/s]

...

  0%|          | 0/3199 [00:00<?, ?it/s]

Evaluation!


  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

Epoch stats: 
{'train_squad_exact_match': 29.603, 'train_squad_f1': 46.5485, 'train_bleu': 0.1703, 'train_meteor': 0.413, 'train_rouge1': 0.4714, 'train_rouge2': 0.353, 'train_rougeL': 0.4658, 'train_rougeLsum': 0.4666, 'train_class_accuracy': 0.633, 'train_class_f1': 0.6291, 'val_squad_exact_match': 27.25, 'val_squad_f1': 44.8266, 'val_bleu': 0.1586, 'val_meteor': 0.388, 'val_rouge1': 0.4554, 'val_rouge2': 0.3165, 'val_rougeL': 0.4511, 'val_rougeLsum': 0.4515, 'val_class_accuracy': 0.6175, 'val_class_f1': 0.6083, 'epoch': 9, 'train_loss': 2.5673603797421873, 'val_loss': 2.668214382948699, 'train_qa_loss': 1.6778174572199294, 'val_qa_loss': 1.7718911038504706, 'train_cls_loss': 0.8895429222329149, 'val_cls_loss': 0.8963232879285459}


In [48]:
model.eval()
start_logits = []
end_logits = []
class_logits = []
accelerator.print("Evaluation!")

for batch in eval_dataloader:
    with torch.no_grad():
        outputs = model(**batch)

    start_logits.append(accelerator.gather(outputs['start_logits']).cpu().numpy())
    end_logits.append(accelerator.gather(outputs['end_logits']).cpu().numpy())
    class_logits.append(accelerator.gather(outputs['classification_logits']).cpu().numpy())

start_logits = np.concatenate(start_logits)
end_logits = np.concatenate(end_logits)
class_logits = np.concatenate(class_logits)
start_logits = start_logits[: len(validation_dataset)]
end_logits = end_logits[: len(validation_dataset)]
class_logits = class_logits[: len(validation_dataset)]

metrics,theoretical_texts,predicted_texts, predicted_classes = compute_metrics(
    start_logits, end_logits,class_logits, validation_dataset, val_data)

Evaluation!


  0%|          | 0/400 [00:00<?, ?it/s]

In [49]:
metrics

{'squad_exact_match': 27.25,
 'squad_f1': 44.8266,
 'bleu': 0.1586,
 'meteor': 0.388,
 'rouge1': 0.4554,
 'rouge2': 0.3165,
 'rougeL': 0.4511,
 'rougeLsum': 0.4515,
 'class_accuracy': 0.6175,
 'class_f1': 0.6083}

In [55]:
for i in range(3):
    val_df_temp = val_df[val_df['labels']==i]
    val_data = Dataset.from_pandas(val_df_temp.reset_index(drop=True), split="val")
    val_dataset = val_data.map(
        preprocess_test_examples,
        batched=True,
        remove_columns=val_data.column_names,
    )
    val_set = val_dataset.remove_columns(["example_id", "offset_mapping"])
    val_set.set_format("torch")
    val_dataloader = DataLoader(
        val_set, collate_fn=default_data_collator, batch_size=32
    )
    eval_dataloader = accelerator.prepare(val_dataloader)

    model.eval()
    start_logits = []
    end_logits = []
    class_logits = []
    accelerator.print("Evaluation!")

    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(accelerator.gather(outputs['start_logits']).cpu().numpy())
        end_logits.append(accelerator.gather(outputs['end_logits']).cpu().numpy())
        class_logits.append(accelerator.gather(outputs['classification_logits']).cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    class_logits = np.concatenate(class_logits)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]
    class_logits = class_logits[: len(validation_dataset)]

    metrics,theoretical_texts,predicted_texts, predicted_classes = compute_metrics(
        start_logits, end_logits,class_logits, val_dataset, val_data)

    print("Class ", label_decoder[i])
    print(metrics)

Map:   0%|          | 0/162 [00:00<?, ? examples/s]

len(inputs["input_ids"]) 276
Evaluation!


  0%|          | 0/162 [00:00<?, ?it/s]

Class  phrase
{'squad_exact_match': 52.4691, 'squad_f1': 67.0443, 'bleu': 0.1605, 'meteor': 0.5652, 'rouge1': 0.6761, 'rouge2': 0.4323, 'rougeL': 0.6743, 'rougeLsum': 0.675, 'class_accuracy': 0.8148, 'class_f1': 0.898}


Map:   0%|          | 0/154 [00:00<?, ? examples/s]

...

Class  passage
{'squad_exact_match': 11.039, 'squad_f1': 35.5833, 'bleu': 0.136, 'meteor': 0.3182, 'rouge1': 0.3639, 'rouge2': 0.3023, 'rougeL': 0.3562, 'rougeLsum': 0.3559, 'class_accuracy': 0.513, 'class_f1': 0.6781}


Map:   0%|          | 0/84 [00:00<?, ? examples/s]

len(inputs["input_ids"]) 242
Evaluation!


  0%|          | 0/84 [00:00<?, ?it/s]

Class  multi
{'squad_exact_match': 8.3333, 'squad_f1': 18.9245, 'bleu': 0.0557, 'meteor': 0.174, 'rouge1': 0.1974, 'rouge2': 0.1189, 'rougeL': 0.1915, 'rougeLsum': 0.1909, 'class_accuracy': 0.4286, 'class_f1': 0.6}


In [57]:
val_data

Dataset({
    features: ['id', 'labels', 'title', 'question', 'context', 'answers'],
    num_rows: 84
})

In [58]:
162+154+84

400

In [ ]:
model.eval()
start_logits = []
end_logits = []
class_logits = []
accelerator.print("Evaluation!")

for batch in tqdm(test_dataloader):
    with torch.no_grad():
        outputs = model(**batch)

    start_logits.append(accelerator.gather(outputs['start_logits']).cpu().numpy())
    end_logits.append(accelerator.gather(outputs['end_logits']).cpu().numpy())
    class_logits.append(accelerator.gather(outputs['classification_logits']).cpu().numpy())

start_logits = np.concatenate(start_logits)
end_logits = np.concatenate(end_logits)
class_logits = np.concatenate(class_logits)
start_logits = start_logits[: len(test_dataset)]
end_logits = end_logits[: len(test_dataset)]
class_logits = class_logits[: len(test_dataset)]

predicted_texts, predicted_classes = compute_metrics(
    start_logits, end_logits, class_logits, test_dataset, test_data, predictOnly=True
)


In [ ]:
df_test_submission = pd.DataFrame(columns=['id','spoilerType'])
df_test_submission['id'] = test_data['id']
df_test_submission['spoilerType'] = predicted_classes
df_test_submission['spoilerType'] = df_test_submission['spoilerType'].apply(lambda x: label_decoder[x])

df_test_submission.to_csv('results_both_robertasquad_new/submission_T1_new.csv',index=False)
df_test_submission.spoilerType.value_counts()

In [ ]:
df_test_submission = pd.DataFrame(columns=['id','spoiler'])
df_test_submission['id'] = test_data['id']
df_test_submission['spoiler'] = predicted_texts
df_test_submission.to_csv('results_both_robertasquad_new/submission_T2_new.csv',index=False)
df_test_submission